# Upload des fichiers préparés vers le S3

La donnée est stockée dans le dossier [01_donnee_source](https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/01_donnee_source/?region=eu-west-3&tab=overview) et l'arborescence simplifiée.

Le chemin où stocker chaque fichier est calculé grâce à la fonction ``calc_dest`` du script ``utils``

In [1]:
def save_donnees_source_to_s3(prep_path):
    """
    Saves prepared data to S3

    args:
        - prep_path : local path of files to send to S3

    returns
        - True in case of success

    """
    
    import utils as u
    import os

    if not 'workbookDir' in globals():
        workbookDir = os.getcwd()
    aws_separator = "/"

    from awsPy.aws_authorization import aws_connector
    from awsPy.aws_s3 import service_s3
    from awsPy.aws_athena import service_athena


    # Connect to S3
    import os
    from pathlib import Path

    bucket = 'calfdata'
    path = os.getcwd()
    parent_path = str(Path(path).parent)
    path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

    con = aws_connector.aws_instantiate(credential = path_cred,
                                            region = 'eu-west-3')
    client= con.client_boto()
    s3 = service_s3.connect_S3(client = client,
                          bucket = 'calfdata')

    # List files to send and compute each destination
    list_files=[]
    for root,dirs,files in os.walk(prep_path):
        for file in files:
              if file.endswith(('.csv')):

                    list_files.append((file, u.calc_dest(filename=file)))

    # Send files to s3
    for (file_to_upload,destination_in_s3) in list_files:
        # if file exist in S3?
        full_path = os.path.join(prep_path, file_to_upload)
        key = aws_separator.join([destination_in_s3,file_to_upload])
        
        # Download only if the file does not already exist
        if not s3.key_exist(key):
            s3.upload_file(full_path, destination_in_s3)
    
    return True
   

In [2]:
# Function call example
prep_path='data/prep4'
save_donnees_source_to_s3(prep_path)

True